# Dashboarding

## Set up
### Set up environment

In [1]:
# Imports
import json
import os
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pycountry

from dash import html, Input, Output, dash_table
from pandas.api.types import is_string_dtype

#JupyterDash.infer_jupyter_proxy_config()

### Load data and helpers

In [2]:
# Helpers
df = pd.read_csv("../data/concatenated/concatenated_all.csv")

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

unabbreviated_countries = [country.strip() for country in df.Country.unique() if country not in countries.keys()]

### Preprocessing

In [3]:
countries["Belgium-Luxembourg"] = countries["Belgium"]
countries["Congo, Democratic Republic"] = countries["Congo, The Democratic Republic of the"]
countries["Czech Republic"] = countries["Czechia"]
countries["German Federal Republic"] = countries["Germany"]
countries["Ireland, Republic of"] = countries["Ireland"]
countries["Korea (Rep. of)"] = countries["Korea, Republic of"]
countries["Russia"] = countries["Russian Federation"]
countries["Soviet Union"] = countries["Russian Federation"]
countries["Taiwan"] = countries["Taiwan, Province of China"]
countries["USA"] = countries["United States"]
countries["Vietnam"] = countries["Viet Nam"]
countries["Zaire"] = countries["Congo, Democratic Republic"]

# Map country names
df["Country code"] = df["Country"].apply(lambda x: countries.get(x, "Unknown code"))

# Convert years to integers
df["Year"] = df["Year"].astype(int)

# Clear out trailing spaces
df = df[df.columns].apply(lambda c: c.str.strip() if is_string_dtype(c) else c)

# Reordering dataset
column_order = ["Country code", "Country", "Activity", "Commodity", "Sub-commodity", "Year", "Amount", "Measure"]
df = df[column_order]

# Remove rows with missing amount
df = df.dropna(subset=["Amount"], axis=0).reset_index(drop=True)

# 
activity = df["Activity"].unique()[0]
commodity = df["Commodity"].unique()[0]

In [4]:
from dash import dcc, html, Input, Output
from jupyter_dash import JupyterDash as Dash

initial_activity = np.sort(df["Activity"])[0]
initial_commodity = np.sort(df["Commodity"])[0]

import pandas as pd
import plotly.express as px

app = Dash(__name__)

#df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                df['Activity'].unique(),
                value=initial_activity,
                id='dropdown_actvity'
            )
        ],
            style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Dropdown(
                df['Commodity'].unique(),
                value=initial_commodity,
                id='dropdown_commodity'
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    
    html.Div([
        dcc.Graph(id='map'),
    ]),
    
    html.Div([
        dcc.Slider(
            min=df['Year'].min(),
            max=df['Year'].max(),
            step=1,
            id='slider_year',
            value=df['Year'].max(),
            marks={str(year): {'label': str(year), 'style': {"transform":"rotate(270deg)"}} for year in df['Year'].unique()}
        )
    ])
])




@app.callback(
    Output('slider_year', 'min'),
    Output('slider_year', 'max'),
    Input('dropdown_actvity', 'value'),
    Input('dropdown_commodity', 'value'))
def update_slider(dropdown_actvity, dropdown_commodity):
    slider_df = df[(df["Activity"]==dropdown_actvity) &
                   (df["Commodity"]==dropdown_commodity)]
    filtered_slider_min = slider_df["Year"].min()
    filtered_slider_max = slider_df["Year"].max()
    return filtered_slider_min, filtered_slider_max
    

@app.callback(
    Output('map', 'figure'),
    Input('dropdown_actvity', 'value'),
    Input('dropdown_commodity', 'value'),
    Input('slider_year', 'value'))
def update_graph(dropdown_actvity, dropdown_commodity,
                 slider_year):
    graph_df = df[(df["Year"]==slider_year) & 
                  (df["Activity"]==dropdown_actvity) &
                  (df["Commodity"]==dropdown_commodity)]

    fig_map = px.choropleth(
        graph_df,
        locations="Country code",
        color="Amount",
        hover_name="Country",
        color_continuous_scale='blugrn',
        height=600,
        title=f"Global {dropdown_actvity} of {dropdown_commodity} in {str(slider_year)}"
    )

    #fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
    return fig_map

app.run_server(host='0.0.0.0', port=8050, mode="jupyterlab", debug=True, threaded=True)
# 127.0.0.1:8050